<a href="https://colab.research.google.com/github/ramasai-badam/Coursera_Capstone/blob/master/speaker_diarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/resemble-ai/Resemblyzer.git

Cloning into 'Resemblyzer'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 595 (delta 2), reused 1 (delta 0), pack-reused 586
Receiving objects: 100% (595/595), 101.46 MiB | 48.23 MiB/s, done.
Resolving deltas: 100% (99/99), done.


In [ ]:
!pip install webrtcvad

     |████████████████████████████████| 71kB 3.2MB/s 
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp36-cp36m-linux_x86_64.whl size=71376 sha256=180f2110f12ffac60f02cf9861ac2677201309ea12ff64866c0f4e3d224fa931
  Stored in directory: /root/.cache/pip/wheels/44/2a/18/bd1aec41cac7c3051fe95d92a6ed446122ea31dc713c432fa1
Successfully built webrtcvad


In [ ]:
!pip install librosa

In [ ]:
!pip install ffmpeg

  Created wheel for ffmpeg: filename=ffmpeg-1.4-cp36-none-any.whl size=6084 sha256=43fd05970d53e39220c13b32f854cd946bcb9f76fc9c1fc423a675e026a72dfc
  Stored in directory: /root/.cache/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
Successfully built ffmpeg


In [ ]:
cd Resemblyzer


/content/Resemblyzer


In [ ]:
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path

#give the file path to your audio file
audio_file_path = "/content/drive/My Drive/threepeople.wav"
wav_fpath = Path(audio_file_path)

wav = preprocess_wav(wav_fpath)
encoder = VoiceEncoder("cpu")
_, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)
print(cont_embeds.shape)

Loaded the voice encoder model on cpu in 0.03 seconds.
(2826, 256)


In [ ]:
!pip install spectralcluster

In [ ]:
from spectralcluster import SpectralClusterer

clusterer = SpectralClusterer(
    min_clusters=3,
    max_clusters=15,
    p_percentile=0.90,
    gaussian_blur_sigma=1)

labels = clusterer.predict(cont_embeds)

In [ ]:
def create_labelling(labels,wav_splits):
    from resemblyzer import sampling_rate
    times = [((s.start + s.stop) / 2) / sampling_rate for s in wav_splits]
    labelling = []
    start_time = 0

    for i,time in enumerate(times):
        if i>0 and labels[i]!=labels[i-1]:
            temp = [str(labels[i-1]),start_time,time]
            labelling.append(tuple(temp))
            start_time = time
        if i==len(times)-1:
            temp = [str(labels[i]),start_time,time]
            labelling.append(tuple(temp))

    return labelling
  
labelling = create_labelling(labels,wav_splits)

In [ ]:
labelling

[('1', 0, 5.84),
 ('0', 5.84, 12.38),
 ('1', 12.38, 20.6),
 ('0', 20.6, 24.26),
 ('1', 24.26, 31.64),
 ('0', 31.64, 32.0),
 ('2', 32.0, 33.86),
 ('0', 33.86, 33.92),
 ('1', 33.92, 33.98),
 ('0', 33.98, 34.1),
 ('1', 34.1, 40.04),
 ('0', 40.04, 47.6),
 ('2', 47.6, 49.28),
 ('0', 49.28, 54.14),
 ('1', 54.14, 54.5),
 ('0', 54.5, 54.86),
 ('1', 54.86, 54.92),
 ('0', 54.92, 55.4),
 ('1', 55.4, 55.46),
 ('0', 55.46, 59.06),
 ('1', 59.06, 66.74),
 ('0', 66.74, 66.86),
 ('1', 66.86, 66.98),
 ('0', 66.98, 76.16),
 ('1', 76.16, 78.08),
 ('0', 78.08, 78.32),
 ('2', 78.32, 81.32),
 ('0', 81.32, 87.86),
 ('1', 87.86, 94.82),
 ('0', 94.82, 97.7),
 ('2', 97.7, 98.96),
 ('0', 98.96, 104.12),
 ('1', 104.12, 109.1),
 ('0', 109.1, 118.76),
 ('1', 118.76, 130.04),
 ('0', 130.04, 134.72),
 ('2', 134.72, 134.84),
 ('0', 134.84, 135.02),
 ('2', 135.02, 136.94),
 ('0', 136.94, 140.18),
 ('1', 140.18, 143.78),
 ('0', 143.78, 152.18),
 ('2', 152.18, 157.94),
 ('1', 157.94, 161.72),
 ('0', 161.72, 170.3)]

In [ ]:
!pip install azure-cognitiveservices-speech

     |████████████████████████████████| 4.6MB 5.6MB/s 


In [ ]:
import azure.cognitiveservices.speech as speechsdk
import time

speech_key, service_region = "c7a2dfb02fb04e229e88d6d73b18d982", "eastus"
speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)

audio_filename = "/content/drive/My Drive/threepeople.wav"
audio_input = speechsdk.AudioConfig(filename=audio_filename)
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)

audio_config = speechsdk.audio.AudioConfig(filename=audio_filename)
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

done = False

def stop_cb(evt):
    print('CLOSING on {}'.format(evt))
    speech_recognizer.stop_continuous_recognition()
    global done
    done=True

speech_recognizer.recognizing.connect(lambda evt: print('RECOGNIZING: {}'.format(evt)))
speech_recognizer.recognized.connect(lambda evt: print('RECOGNIZED: {}'.format(evt)))
speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))
speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
# speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))

speech_recognizer.session_stopped.connect(stop_cb)
speech_recognizer.canceled.connect(stop_cb)

speech_recognizer.start_continuous_recognition()
while not done:
    time.sleep(.5)

SESSION STARTED: SessionEventArgs(session_id=3723bd2f928e4abe8bcd29f4abba97ba)
RECOGNIZING: SpeechRecognitionEventArgs(session_id=3723bd2f928e4abe8bcd29f4abba97ba, result=SpeechRecognitionResult(result_id=1630b11383ba4646a42e0854e230df45, text="hey", reason=ResultReason.RecognizingSpeech))
RECOGNIZING: SpeechRecognitionEventArgs(session_id=3723bd2f928e4abe8bcd29f4abba97ba, result=SpeechRecognitionResult(result_id=575f0e460ae645d7a1216b28dc4644dd, text="hey gene", reason=ResultReason.RecognizingSpeech))
RECOGNIZING: SpeechRecognitionEventArgs(session_id=3723bd2f928e4abe8bcd29f4abba97ba, result=SpeechRecognitionResult(result_id=e57b5c1cccb54aaa916067b4dc242397, text="hey jean", reason=ResultReason.RecognizingSpeech))
RECOGNIZING: SpeechRecognitionEventArgs(session_id=3723bd2f928e4abe8bcd29f4abba97ba, result=SpeechRecognitionResult(result_id=1e3d7198b31e4d0ab82179358100cfbe, text="hey jean", reason=ResultReason.RecognizingSpeech))
RECOGNIZING: SpeechRecognitionEventArgs(session_id=3723bd2

In [ ]:
#used to trim audio and video.

import ffmpeg 

#first finding out the length of the list to easily iterate over it.

# print(len(labelling))

#trim entire audio into segments based on end and start time

for index in range(len(labelling)) : 

  print(labelling[index][1])

In [ ]:
!git clone https://github.com/tyiannak/pyAudioAnalysis.git

Cloning into 'pyAudioAnalysis'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 2399 (delta 22), reused 10 (delta 5), pack-reused 2359
Receiving objects: 100% (2399/2399), 134.59 MiB | 30.52 MiB/s, done.
Resolving deltas: 100% (1482/1482), done.


In [ ]:
!pip install -r ./requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: './requirements.txt'


In [ ]:
cd pyAudioAnalysis/

/content/pyAudioAnalysis


In [ ]:
!pip install -r ./requirements.txt

     |████████████████████████████████| 13.1MB 242kB/s 
     |████████████████████████████████| 81kB 12.8MB/s 
     |████████████████████████████████| 20.2MB 1.4MB/s 
     |████████████████████████████████| 153kB 51.0MB/s 
     |████████████████████████████████| 153kB 52.3MB/s 
     |████████████████████████████████| 6.7MB 52.2MB/s 
     |████████████████████████████████| 61kB 9.7MB/s 
  Created wheel for simplejson: filename=simplejson-3.16.0-cp36-cp36m-linux_x86_64.whl size=114013 sha256=cda5b194610bc15af4d60e9ac833397eccc808791e9a5bb53107d530e0efdc28
  Stored in directory: /root/.cache/pip/wheels/5d/1a/1e/0350bb3df3e74215cd91325344cc86c2c691f5306eb4d22c77
  Created wheel for hmmlearn: filename=hmmlearn-0.2.2-cp36-cp36m-linux_x86_64.whl size=323264 sha256=6ee4670611325526a7449f606496a92bcd1a76d97267db934f03451c1f9a4924
  Stored in directory: /root/.cache/pip/wheels/2c/b6/0e/63a865a30e21e01d04f417d8995fbfb793d6bd464707efc546
Successfully built simplejson hmmlearn
ERROR: datascience 0.

In [ ]:
!pip install -e .

Obtaining file:///content/pyAudioAnalysis
  Running setup.py develop for pyAudioAnalysis


In [ ]:
python audioAnalysis.py speakerDiarization -i /content/drive/My Drive/threepeople.wav --num 0

SyntaxError: ignored

In [ ]:
python audioAnalysis.py speakerDiarization -i /content/drive/My Drive/threepeople.wav --num 0

SyntaxError: ignored

In [ ]:
import audioAnalysis



ModuleNotFoundError: ignored